# Probability

An intro to **probability** in Clojure

## `P` function

`P` is the name for the *Probability* function

In [1]:
(ns probability
  (:require [clojure.math.combinatorics :as combinatorics]))

In [2]:
(defn P
  "Probability of an event occurring"
  [event space]
  (/ (count (filter (set event) space))
     (count space)))

#'probability/P

In [3]:
(def D [1 2 3 4 5 6])
(def even [2 4 6])

#'probability/even

In [4]:
(P even D)

1/2

We might have done `(/ (count event) (count space))`, but in this way we would have counted stuff not present in the event space.

In [5]:
(def even [2 4 6 8 10 12])

(P even D)

1/2

## Urn problems

Usually urns are used to explain probability. Let's take this problem as an example:

> An urn contains 23 balls: 8 white, 6 blue, and 9 red. We select six balls at random (each possible selection is equally likely). What is the probability of each of these possible outcomes:
1. all balls are red
2. 3 are blue, 2 are white, and 1 is red
3. exactly 4 balls are white

Here an *outcome* is a set of 6 balls and the *sample space* is the set of all possible 6 ball combinations. We can solve this problem with our `P` function and *counting*.

Note that an outcome is a *set* of balls, not a *sequence*, so the order doesn't matter. To account for the fact that we have more than one ball of the same color, we will represent them with the initial followed by a number, such as: `W1`

In [6]:
(defn cross
  "Create a set of ways of concatenating
  items from A with items from B"
  [A B]
  (set (map #(str A %) B)))

#'probability/cross

In [7]:
(def balls [{:A "W" :B (range 1 9)}
            {:A "B" :B (range 1 7)}
            {:A "R" :B (range 1 10)}])

#'probability/balls

In [8]:
(def urn (mapcat #(cross (:A %) (:B %)) balls))

#'probability/urn

In [9]:
(count urn)

23

By using `clojure.math.combinatorics` (here required in the `ns` declaration in the first cell as `combinatorics`) we can create the sample space as the set of all 6 balls combinations.

In [10]:
(defn combos
  "All combinations of n items, each one concatenated
  with str"
  [items n]
  (let [combi (combinatorics/combinations items n)]
    (map #(clojure.string/join " " %) combi)))

#'probability/combos

In [11]:
(def U6 (combos urn 6))

(count U6)

100947

In [12]:
(clojure.pprint/pprint (take 10 (random-sample 0.5 U6)))

("W3 W7 W6 W1 W8 W5"
 "W3 W7 W6 W1 W8 W2"
 "W3 W7 W6 W1 W8 B6"
 "W3 W7 W6 W1 W8 R4"
 "W3 W7 W6 W1 W8 R8"
 "W3 W7 W6 W1 W8 R6"
 "W3 W7 W6 W1 W8 R2"
 "W3 W7 W6 W1 W8 R9"
 "W3 W7 W6 W1 W8 R1"
 "W3 W7 W6 W1 W5 W2")


To check if it is true that 100,947 is the right number of possible combinations we can build a function that calculates it

In [13]:
(defn ! [n]
  (loop [cur n 
         acc 1]
    (if (zero? cur) acc
      (recur (dec cur) (*' cur acc)))))

(defn choose
  "Number of ways to choose c items from
  a list of n items"
  [n c]
  (int (/ (! n) 
          (* (! (- n c)) (! c)))))

#'probability/choose

In [14]:
(choose 23 6)

100947

### Problem 1: what's the probability of selecting 6 red balls?

In [15]:
(defn count-color
  [string color]
  (->> string
       (re-seq (re-pattern color))
       count))

(defn draw
  [urn color n]
  (filter 
    #(= (count-color % color) n) 
    urn))

(def red6 (draw U6 "R" 6))

#'probability/red6

In [16]:
(P red6 U6)

4/4807

In [17]:
(count red6)

84

In [18]:
(choose 9 6)

84

As we saw there are **84** different ways of drawing 6 red balls. Since there are 9 red balls in the urn, we are asking `(choose 9 6)` basically. This means that **P** of 6 red balls is 9 choose 6 divided by the size of the sample space.

In [19]:
(= (P red6 U6)
   (/ (choose 9 6)
      (count U6)))

true

### Problem 2: what is the probability of 3 blue, 2 white and 1 red?

In [20]:
(defn draw
  [urn colors n]
  (let [count-colors (fn [s c]
                       (map #(count-color s %) c))
        right-draw? (fn [s c n]
                      (= (count-colors s c)
                         n))]
    (filter #(right-draw? % colors n) urn)))

#'probability/draw

In [21]:
(def b3w2r1 (draw U6 ["B" "W" "R"] [3 2 1]))

(P b3w2r1 U6)

240/4807

In [22]:
(= (P b3w2r1 U6)
   (/ (* (choose 6 3)
         (choose 8 2)
         (choose 9 1))
      (count U6)))

true

### Problem 3: what is the probability of drawing 4 white balls?

In [23]:
(def w4 (draw U6 ["W"] [4]))

(P w4 U6)

350/4807

## Generalizing `P`

In [24]:
(defn P
  [event space]
  (let [e (if (fn? event)
            (filter event space)
            (filter (set event) space))]
    (/ (count e)
       (count space))))

#'probability/P

In [25]:
(P even? D)

1/2

In [26]:
(def D12 (range 1 13))

(P even? D12)

1/2

In [27]:
(defn divisible? 
  [a b]
  (zero? (mod a b)))

(defn prime? 
  [n]
  (and (> n 1) (not-any? (partial divisible? n) (range 2 n))))

#'probability/prime?

In [28]:
(def D3 
  (->> (for [a D
             b D
             c D]
         [a b c])
       (map #(reduce + %))))

#'probability/D3

In [29]:
(P prime? D3)

73/216

## Card Problems

Let's play some Poker! We define a `deck` as a set of 52 cards, and `hands` as the sample space of all combinations of 5 cards:

In [30]:
(def ranks (clojure.string/split "A23456789TJQK" #""))

(def suits&ranks
  [{:A "S" :B ranks}
   {:A "H" :B ranks}
   {:A "D" :B ranks}
   {:A "C" :B ranks}])

#'probability/suits&ranks

In [31]:
(def deck (atom 
            (mapcat #(cross (:A %) (:B %)) suits&ranks)))

#'probability/deck

In [32]:
(count @deck)

52

In [33]:
(def hands (atom (combos @deck 5)))

#'probability/hands

In [34]:
(= (count @hands)
   (choose 52 5))

true

In [35]:
(count @hands)

2598960

In [36]:
(clojure.pprint/pprint 
  (take 10 (random-sample 0.5 @hands)))

("S9 S6 S2 S5 SA"
 "S9 S6 S2 S5 SJ"
 "S9 S6 S2 S5 SQ"
 "S9 S6 S2 S5 S4"
 "S9 S6 S2 S5 H9"
 "S9 S6 S2 S5 HQ"
 "S9 S6 S2 S5 H6"
 "S9 S6 S2 S5 H8"
 "S9 S6 S2 S5 H4"
 "S9 S6 S2 S5 HA")


In [37]:
(defn flush?
  [hand]
  (let [suits (map :A suits&ranks)]
    (some true?
          (map #(= (count-color hand %) 5) suits))))

#'probability/flush?

In [38]:
(P flush? @hands)

33/16660

In [39]:
(defn four-kind?
  [hand]
  (some true?
        (map #(= (count-color hand %) 4) ranks)))

#'probability/four-kind?

In [40]:
(P four-kind? @hands)

1/4165

## Gambling, Triangles and the birth of Probability

In [41]:
(defn continuations
  [H T]
  (let [rounds (map #(clojure.string/split % #"") (repeat (- (+ H T) 1) "ht"))]
    (apply combinatorics/cartesian-product rounds)))

#'probability/continuations

In [52]:
(defn win-probability
  [H T]
  (let [hwins (fn [out]
                (>= (count
                      (filter
                        #(= "h" %) out)) H))]
   (P hwins (continuations H T))))

#'probability/win-probability

In [53]:
(continuations 2 3)

(("h" "h" "h" "h") ("h" "h" "h" "t") ("h" "h" "t" "h") ("h" "h" "t" "t") ("h" "t" "h" "h") ("h" "t" "h" "t") ("h" "t" "t" "h") ("h" "t" "t" "t") ("t" "h" "h" "h") ("t" "h" "h" "t") ("t" "h" "t" "h") ("t" "h" "t" "t") ("t" "t" "h" "h") ("t" "t" "h" "t") ("t" "t" "t" "h") ("t" "t" "t" "t"))

In [55]:
(win-probability 2 3)

11/16

## Probability Distributions

In [62]:
(defn make-prob-dist
  [probs]
  (let [values (vals probs)
        total  (apply + values)
        dist (zipmap 
               (keys probs) 
               (map #(/ % total) values))]
    (assert (every? #(>= % 0) (vals dist)))
    dist))

#'probability/make-prob-dist

In [110]:
(defn P
  [event space]
  (cond
    (and (fn? event)
         (map? space)) (->> (filter event (keys space))
                            (select-keys space)
                            vals
                            (apply +))
    (fn? event) (/ (count (filter event space))
                   (count space))
    :else (/ (count (filter (set event) space))
             (count space))))

#'probability/P

In [111]:
(def DK
  (make-prob-dist {"GG" 121801
                   "GB" 126840
                   "BG" 127123
                   "BB" 135138}))

DK

{"GG" 121801/510902, "GB" 9060/36493, "BG" 127123/510902, "BB" 67569/255451}

In [137]:
(defn make-pred
  [pos sex]
  (let [pos (if (= 0 pos)
              [0 1]
              [pos])]
    (fn [k] 
      (= (apply subs k pos) sex))))

#'probability/make-pred

In [139]:
(def first-girl?
  (make-pred 0 "G"))

#'probability/first-girl?

In [141]:
(P first-girl? DK)

248641/510902

In [142]:
(P (make-pred 0 "G") DK)

248641/510902

In [143]:
(P (make-pred 0 "B") DK)

262261/510902

In [145]:
(P (make-pred 1 "G") (filter first-girl? (keys DK)))

1/2

In [146]:
(P (make-pred 1 "G") (filter (make-pred 0 "B") (keys DK)))

1/2